In [85]:
# Import necessary libraries
import sqlite3
import pandas as pd

# Establish database connection
conn = sqlite3.connect('CarSharing.db') 

# Step 1: Drop the existing CarSharing table if it exists
drop_table_query = "DROP TABLE IF EXISTS CarSharing;"
conn.execute(drop_table_query)

# Step 2: Create table schema
create_table_query = """
CREATE TABLE CarSharing (
    id INTEGER PRIMARY KEY,
    timestamp TEXT,
    season INTEGER,
    holiday INTEGER,
    workingday INTEGER,
    weather INTEGER,
    temp REAL,
    temp_feel REAL,
    humidity REAL,
    windspeed REAL,
    demand INTEGER
);"""
conn.execute(create_table_query)

# Step 3: Import data into the created table named "CarSharing"
csv_file_path = r'C:\Users\Jyothesh karnam\Desktop\dadb\CarSharing.csv'
car_data = pd.read_csv(csv_file_path)  # Load data from CSV into a Data-Frame
car_data.to_sql('CarSharing', conn, if_exists='replace', index=False)  # Write DataFrame to SQLite table

# Step 4: Check if backup table exists, drop it if it does
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS CarSharingBackup;")
cursor.close()

# Create a backup table and copy data
backup_query = "CREATE TABLE CarSharingBackup AS SELECT * FROM CarSharing;"
conn.execute(backup_query)

# Commit changes
conn.commit()

# Fetch the table into a pandas DataFrame
query = "SELECT * FROM CarSharing;"
carsharing_data = pd.read_sql(query, conn)

# Convert timestamp column to datetime format and format it to display both date and time
carsharing_data['timestamp'] = pd.to_datetime(carsharing_data['timestamp']).dt.strftime('%d/%m/%Y %H:%M')

# Close the database connection
conn.close()

# Apply the center alignment style to each individual cell in the DataFrame
styled_data = carsharing_data.head().style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
styled_data



,id,timestamp,season,holiday,workingday,weather,temp,temp_feel,humidity,windspeed,demand
0,1,01/01/2017 00:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,81.000000,0.000000,2.772589
1,2,01/01/2017 01:00,spring,No,No,Clear or partly cloudy,9.020000,13.635000,80.000000,0.000000,3.688879
2,3,01/01/2017 02:00,spring,No,No,Clear or partly cloudy,9.020000,13.635000,80.000000,0.000000,3.465736
3,4,01/01/2017 03:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,75.000000,0.000000,2.564949
4,5,01/01/2017 04:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,75.000000,0.000000,0.000000


In [86]:
import sqlite3
import pandas as pd

# Establish database connection
conn = sqlite3.connect('CarSharing.db')
cur = conn.cursor()  # Create a cursor object

# Add a new column temp_category to the CarSharing table with a specified maximum length of 3 characters
try:
    cur.execute("ALTER TABLE CarSharing ADD COLUMN temp_category TEXT(3);")
except sqlite3.OperationalError:
    print("Column 'temp_category' already exists.")

# Update the values in the temp_category column using a CASE ex-pression
update_query = '''
UPDATE CarSharing
SET temp_category = (
    CASE
        WHEN temp_feel < 10 THEN 'Cold'
        WHEN temp_feel BETWEEN 10 AND 25 THEN 'Mild'
        ELSE 'Hot'
    END
);
'''
cur.execute(update_query)

# Commit the changes to the database
conn.commit()

# Close the cursor|
cur.close()

query = "SELECT * FROM CarSharing;"
data = pd.read_sql(query, conn)
# Close the connection after fetching the data
conn.close()

# Convert timestamp column to datetime format and format it to display both date and time
# data['timestamp'] = pd.to_datetime(data['timestamp']).dt.strftime('%d/%m/%Y %H:%M')

# Apply the center alignment style to each individual cell in the DataFrame
styled_data = data.head().style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
styled_data



,id,timestamp,season,holiday,workingday,weather,temp,temp_feel,humidity,windspeed,demand,temp_category
0,1,2017-01-01 00:00:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,81.000000,0.000000,2.772589,Mild
1,2,2017-01-01 01:00:00,spring,No,No,Clear or partly cloudy,9.020000,13.635000,80.000000,0.000000,3.688879,Mild
2,3,2017-01-01 02:00:00,spring,No,No,Clear or partly cloudy,9.020000,13.635000,80.000000,0.000000,3.465736,Mild
3,4,2017-01-01 03:00:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,75.000000,0.000000,2.564949,Mild
4,5,2017-01-01 04:00:00,spring,No,No,Clear or partly cloudy,9.840000,14.395000,75.000000,0.000000,0.000000,Mild


In [87]:
import sqlite3
import pandas as pd  # Import Pandas for data manipulation

# Establish a connection to the database
conn = sqlite3.connect('CarSharing.db')
cursor = conn.cursor()

# Ensure the 'CarSharing' table contains the 'temp_category' column
cursor.execute("PRAGMA table_info(CarSharing);")
columns_info = cursor.fetchall()
column_names = [info[1] for info in columns_info]

if 'temp_category' not in column_names:
    # Add 'temp_category' column if it doesn't exist
    cursor.execute("ALTER TABLE CarSharing ADD COLUMN temp_category TEXT;")

# Check if the 'temperature' table exists, drop it if it does to start fresh
cursor.execute("DROP TABLE IF EXISTS temperature;")

# Create the 'temperature' table
cursor.execute("""
CREATE TABLE temperature (
    temp REAL,
    temp_feel REAL,
    temp_category TEXT
);
""")

# Insert data into the 'temperature' table from the 'CarSharing' table
cursor.execute("""
INSERT INTO temperature (temp, temp_feel, temp_category)
SELECT temp, temp_feel, temp_category FROM CarSharing;
""")

# Commit the changes to the database
conn.commit()

# Display the contents of the 'temperature' table to verify
cursor.execute("SELECT * FROM temperature LIMIT 5;")
temperature_data = cursor.fetchall()
df_temperature_data = pd.DataFrame(temperature_data, columns=[desc[0] for desc in cursor.description])

# Print the DataFrame for temperature data with styling
styled_temperature_data = df_temperature_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])
print("Temperature Data:")
display(styled_temperature_data)

# Create a new table excluding 'temp' and 'temp_feel' columns
cursor.execute("""
CREATE TABLE IF NOT EXISTS CarSharing_temp AS 
SELECT id, timestamp, season, holiday, workingday, weather, 
       humidity, windspeed, demand, temp_category
FROM CarSharing;
""")

# Drop the original CarSharing table
cursor.execute("DROP TABLE CarSharing;")

# Rename the temporary table to CarSharing
cursor.execute("ALTER TABLE CarSharing_temp RENAME TO CarSharing;")

# Commit the changes to the database
conn.commit()

# Display the contents of the updated CarSharing table

cursor.execute("SELECT * FROM CarSharing LIMIT 5;")
updated_carsharing_data = cursor.fetchall()
df_updated_carsharing_data = pd.DataFrame(updated_carsharing_data, columns=[desc[0] for desc in cursor.description])

# Print the DataFrame for updated CarSharing data with styling
styled_updated_carsharing_data = df_updated_carsharing_data.style.set_properties(**{'text-align': 'cen-ter'}).set_table_styles([{'selector': 'th', 'props': [('text-align', 'center')]}])
print("\nUpdated CarSharing Data:")
display(styled_updated_carsharing_data)

# Close the cursor and connection to clean up
cursor.close()
conn.close()


Temperature Data:


,temp,temp_feel,temp_category
0,9.840000,14.395000,Mild
1,9.020000,13.635000,Mild
2,9.020000,13.635000,Mild
3,9.840000,14.395000,Mild
4,9.840000,14.395000,Mild



Updated CarSharing Data:


,id,timestamp,season,holiday,workingday,weather,humidity,windspeed,demand,temp_category
0,1,2017-01-01 00:00:00,spring,No,No,Clear or partly cloudy,81.000000,0.000000,2.772589,Mild
1,2,2017-01-01 01:00:00,spring,No,No,Clear or partly cloudy,80.000000,0.000000,3.688879,Mild
2,3,2017-01-01 02:00:00,spring,No,No,Clear or partly cloudy,80.000000,0.000000,3.465736,Mild
3,4,2017-01-01 03:00:00,spring,No,No,Clear or partly cloudy,75.000000,0.000000,2.564949,Mild
4,5,2017-01-01 04:00:00,spring,No,No,Clear or partly cloudy,75.000000,0.000000,0.000000,Mild


In [88]:
import sqlite3
import pandas as pd

# Establish database connection
conn = sqlite3.connect('CarSharing.db')

# Step 1: Create a temporary table with unique weather codes
conn.execute("""
CREATE TEMP TABLE IF NOT EXISTS WeatherCodes AS
SELECT DISTINCT
    weather,
    ROW_NUMBER() OVER (ORDER BY weather) AS code
FROM CarSharing;
""")

# Step 2: Add the 'weather_code' column to the 'CarSharing' table if it doesn't exist
try:
    conn.execute("ALTER TABLE CarSharing ADD COLUMN weather_code INTEGER;")
except sqlite3.OperationalError:
    # If the column already exists, this error will be caught
    print("Column 'weather_code' already exists.")

# Step 3: Update the 'weather_code' column with the corresponding code for each weather value
update_query = """
UPDATE CarSharing
SET weather_code = (
    SELECT code
    FROM WeatherCodes
    WHERE CarSharing.weather = WeatherCodes.weather
);
"""
conn.execute(update_query)

# Commit the changes to the database
conn.commit()

# Optional: Display rows to verify the new 'weather_code' column
query = "SELECT * FROM CarSharing LIMIT 5;"
updated_data = pd.read_sql(query, conn)

conn.close()

# Apply the center alignment style to each individual cell in the DataFrame
styled_updated_data = updated_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
styled_updated_data



,id,timestamp,season,holiday,workingday,weather,humidity,windspeed,demand,temp_category,weather_code
0,1,2017-01-01 00:00:00,spring,No,No,Clear or partly cloudy,81.000000,0.000000,2.772589,Mild,1
1,2,2017-01-01 01:00:00,spring,No,No,Clear or partly cloudy,80.000000,0.000000,3.688879,Mild,1
2,3,2017-01-01 02:00:00,spring,No,No,Clear or partly cloudy,80.000000,0.000000,3.465736,Mild,1
3,4,2017-01-01 03:00:00,spring,No,No,Clear or partly cloudy,75.000000,0.000000,2.564949,Mild,1
4,5,2017-01-01 04:00:00,spring,No,No,Clear or partly cloudy,75.000000,0.000000,0.000000,Mild,1


In [89]:


# # Import necessary libraries
# import sqlite3
# import pandas as pd

# # Establish database connection
# conn = sqlite3.connect('CarSharing.db')

# # Step 1: Create the 'weather' table
# create_weather_table_query = """
# CREATE TABLE IF NOT EXISTS weather (
#     weather TEXT,
#     weather_code INTEGER PRIMARY KEY
# );
# """

# conn.execute(create_weather_table_query)

# # Step 2: Insert distinct weather and weather_code pairs into the 'weather' table
# # Note: Using INSERT OR IGNORE to avoid duplicate insertion errors
# insert_weather_data_query = """
# INSERT OR IGNORE INTO weather (weather, weather_code)
# SELECT DISTINCT weather, weather_code FROM CarSharing;
# """

# conn.execute(insert_weather_data_query)

# # Commit the changes to the database
# conn.commit()

# # Display the contents of the 'weather' table to verify
# query = "SELECT * FROM weather;"
# weather_data = pd.read_sql(query, conn)

# # Apply the center alignment style to each individual cell in the DataFrame
# styled_weather_data = weather_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
#     'selector': 'th',  # Apply to all headers
#     'props': [('text-align', 'center')]
# }])

# # Display the styled DataFrame
# display(styled_weather_data)

# # Close the connection to proceed with modifications
# conn.close()

# # Re-establish database connection to proceed with modifications
# conn = sqlite3.connect('CarSharing.db')

# # Create a temporary table that excludes the 'weather' and 'temp_category' columns
# create_temp_carsharing_query = """
# CREATE TABLE IF NOT EXISTS CarSharing_temp AS 
# SELECT id, timestamp, season, holiday, workingday, humidity, 
#        windspeed, demand, temp_category, weather_code
# FROM CarSharing;
# """
# conn.execute(create_temp_carsharing_query)

# # Drop the original CarSharing table
# conn.execute("DROP TABLE CarSharing;")

# # Rename the temporary table to CarSharing
# conn.execute("ALTER TABLE CarSharing_temp RENAME TO CarSharing;")

# # Commit the changes to the database
# conn.commit()

# # Display the updated CarSharing table
# print("\nUpdated CarSharing Table:")
# updated_carsharing_data = pd.read_sql("SELECT * FROM CarSharing LIMIT 5;", conn)

# # Apply the center alignment style to each individual cell in the DataFrame
# styled_updated_carsharing_data = updated_carsharing_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

# # Display the styled DataFrame
# display(styled_updated_carsharing_data)

# # Close the database connection
# conn.close()


# Import necessary libraries
import sqlite3
import pandas as pd

# Establish database connection
conn = sqlite3.connect('CarSharing.db')

# Step 1: Create the 'weather' table
create_weather_table_query = """
CREATE TABLE IF NOT EXISTS weather (
    weather TEXT,
    weather_code INTEGER PRIMARY KEY
);
"""

conn.execute(create_weather_table_query)

# Step 2: Insert distinct weather and weather_code pairs into the 'weather' table
# Note: Using INSERT OR IGNORE to avoid duplicate insertion errors
insert_weather_data_query = """
INSERT OR IGNORE INTO weather (weather, weather_code)
SELECT DISTINCT weather, weather_code FROM CarSharing;
"""

conn.execute(insert_weather_data_query)

# Commit the changes to the database
conn.commit()

# Display the contents of the 'weather' table to verify
query = "SELECT * FROM weather;"
weather_data = pd.read_sql(query, conn)

# Apply the center alignment style to each individual cell in the DataFrame
styled_weather_data = weather_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',  # Apply to all headers
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
display(styled_weather_data)

# Close the connection to proceed with modifications
conn.close()

# Re-establish database connection to proceed with modifications
conn = sqlite3.connect('CarSharing.db')

# Create a temporary table that excludes the 'weather' and 'temp_category' columns
create_temp_carsharing_query = """
CREATE TABLE IF NOT EXISTS CarSharing_temp AS 
SELECT id, timestamp, season, holiday, workingday, humidity, 
       windspeed, demand, temp_category, weather_code
FROM CarSharing;
"""
conn.execute(create_temp_carsharing_query)

# Drop the original CarSharing table
conn.execute("DROP TABLE CarSharing;")

# Rename the temporary table to CarSharing
conn.execute("ALTER TABLE CarSharing_temp RENAME TO CarSharing;")

# Commit the changes to the database
conn.commit()

# Display the updated CarSharing table
print("\nUpdated CarSharing Table:")
updated_carsharing_data = pd.read_sql("SELECT * FROM CarSharing LIMIT 5;", conn)

# Apply the center alignment style to each individual cell in the DataFrame
styled_updated_carsharing_data = updated_carsharing_data.style.set_properties(**{'text-align': 'center'}).set_table_styles([dict(selector='th', props=[('text-align', 'center')])])

# Display the styled DataFrame
display(styled_updated_carsharing_data)

# Close the database connection
conn.close()


,weather_code,weather
0,1,Clear or partly cloudy
1,2,Mist
2,3,Light snow or rain
3,4,heavy rain/ice pellets/snow + fog
4,5859,Light snow or rain
5,6566,Mist
6,8708,heavy rain/ice pellets/snow + fog



Updated CarSharing Table:


,id,timestamp,season,holiday,workingday,humidity,windspeed,demand,temp_category,weather_code
0,1,2017-01-01 00:00:00,spring,No,No,81.000000,0.000000,2.772589,Mild,1
1,2,2017-01-01 01:00:00,spring,No,No,80.000000,0.000000,3.688879,Mild,1
2,3,2017-01-01 02:00:00,spring,No,No,80.000000,0.000000,3.465736,Mild,1
3,4,2017-01-01 03:00:00,spring,No,No,75.000000,0.000000,2.564949,Mild,1
4,5,2017-01-01 04:00:00,spring,No,No,75.000000,0.000000,0.000000,Mild,1


In [90]:
# TASK 6
# Import necessary libraries
import sqlite3
import pandas as pd

# Establish database connection
conn = sqlite3.connect('CarSharing.db')
cur = conn.cursor()

# Drop the existing "time" table to avoid schema mismatch issues. Use with caution!
cur.execute('DROP TABLE IF EXISTS time;')

# Create the table named "time" with the correct columns
cur.execute('''
CREATE TABLE time(
    timestamp TEXT,
    hour INTEGER,
    weekday TEXT,
    month TEXT);
''')

# Insert rows into the table "time" from "CarSharing" table, with transformations
# Exclude weekends (Saturday and Sunday) from the inserted data
cur.execute('''
INSERT INTO time (timestamp, hour, weekday, month)
SELECT
    strftime('%Y-%m-%d %H:%M:%S', timestamp) as timestamp,
    strftime('%H', timestamp) as hour,
    CASE strftime('%w', timestamp)
        WHEN '1' THEN 'Monday'
        WHEN '2' THEN 'Tuesday'
        WHEN '3' THEN 'Wednesday'
        WHEN '4' THEN 'Thursday'
        WHEN '5' THEN 'Friday'
        ELSE NULL
    END as weekday,
    CASE strftime('%m', timestamp)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END as month
FROM CarSharing
WHERE strftime('%w', timestamp) NOT IN ('0', '6');
''')

# Commit the changes to the database
conn.commit()

# Fetch and print the data structure of the "time" table using Pandas
columns_df = pd.read_sql_query("PRAGMA table_info(time);", conn)
print("Table 'time' columns:", ", ".join(columns_df['name'].tolist()))

# Query and display the first 5 rows of the "time" table using Pandas
time_df = pd.read_sql_query("SELECT * FROM time LIMIT 5;", conn)

# Apply center alignment for the content in the cells and headers
time_df_styled = time_df.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print("\nFirst 5 rows of the 'time' table:")
display(time_df_styled)

# Close the database connection
conn.close()


Table 'time' columns: timestamp, hour, weekday, month

First 5 rows of the 'time' table:


,timestamp,hour,weekday,month
0,2017-01-02 00:00:00,0,Monday,January
1,2017-01-02 01:00:00,1,Monday,January
2,2017-01-02 02:00:00,2,Monday,January
3,2017-01-02 03:00:00,3,Monday,January
4,2017-01-02 04:00:00,4,Monday,January


In [20]:
# TASK 7 subtask A

import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('CarSharing.db')
cursor = conn.cursor()

# Step 1: Create a table to store the highest demand data (drop if exists for re-runs)
cursor.execute('''
DROP TABLE IF EXISTS HighestDemand2017;
''')
cursor.execute('''
CREATE TABLE HighestDemand2017 (
    timestamp TEXT,
    max_demand INTEGER
);
''')

# Your existing query with slight modification for grouping and ordering
query_highest_demand = """
SELECT timestamp, MAX(demand) AS max_demand
FROM CarSharing
WHERE 
    strftime('%Y', timestamp) = '2017'
    AND strftime('%w', timestamp) NOT IN ('0', '6')  -- Exclude Sundays and Saturdays
GROUP BY timestamp
ORDER BY MAX(demand) DESC
LIMIT 1;
"""

# Execute the query and fetch the result
highest_demand_data = pd.read_sql(query_highest_demand, conn)

# Check if there is any data fetched
if not highest_demand_data.empty:
    # Step 2: Insert the fetched data into the new table
    for _, row in highest_demand_data.iterrows():
        cursor.execute('''
        INSERT INTO HighestDemand2017 (timestamp, max_demand) 
        VALUES (?, ?);
        ''', (row['timestamp'], row['max_demand']))
    
    # Step 3: Commit the insert operations to save changes
    conn.commit()
    
    # Fetch and display the result from the new table
    highest_demand_result = pd.read_sql('SELECT * FROM HighestDemand2017', conn)
    
    # Apply the center alignment style and hide the index
    styled_highest_demand_result = highest_demand_result.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
        'selector': 'th',  # Apply to all headers
        'props': [('text-align', 'center')]
    }]).hide(axis='index')
    
    print("Date and Time with Highest Demand Rate in 2017 (Excluding Weekends):")
    display(styled_highest_demand_result)
else:
    print("No data found for the specified criteria.")

# Don't forget to close your database connection when done
conn.close()


Date and Time with Highest Demand Rate in 2017 (Excluding Weekends):


timestamp,max_demand
2017-06-15 17:00:00,6.458338


In [21]:
# # # # TASK 7 subtask B


import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS highest_lowest_avg_demand;")

# Create the table with specific column types, including the new demand_type column
c.execute("""
CREATE TABLE highest_lowest_avg_demand (
    weekday TEXT,
    month TEXT,
    season TEXT,
    avg_demand REAL,
    demand_type TEXT
);
""")

# Calculate the average demand for each combination of weekday, month, and season in 2017, excluding weekends
avg_demand_query = """
SELECT 
    CASE strftime('%w', timestamp) 
        WHEN '1' THEN 'Monday' 
        WHEN '2' THEN 'Tuesday' 
        WHEN '3' THEN 'Wednesday' 
        WHEN '4' THEN 'Thursday' 
        WHEN '5' THEN 'Friday' 
    END AS weekday,
    CASE strftime('%m', timestamp) 
        WHEN '01' THEN 'January' 
        WHEN '02' THEN 'February' 
        WHEN '03' THEN 'March' 
        WHEN '04' THEN 'April' 
        WHEN '05' THEN 'May' 
        WHEN '06' THEN 'June' 
        WHEN '07' THEN 'July' 
        WHEN '08' THEN 'August' 
        WHEN '09' THEN 'September' 
        WHEN '10' THEN 'October' 
        WHEN '11' THEN 'November' 
        WHEN '12' THEN 'December' 
    END AS month,
    season,
    AVG(demand) AS avg_demand
FROM CarSharing
WHERE 
    strftime('%Y', timestamp) = '2017'
    AND strftime('%w', timestamp) NOT IN ('0', '6')
GROUP BY weekday, month, season
ORDER BY avg_demand DESC
"""
c.execute(avg_demand_query)
avg_demand_details_2017 = c.fetchall()

# Convert the query results into a DataFrame
df_avg_demand_details_2017 = pd.DataFrame(avg_demand_details_2017, columns=['weekday', 'month', 'season', 'avg_demand'])

# Identify the highest and lowest average demand rates
highest_avg_demand = df_avg_demand_details_2017.iloc[[0]].assign(demand_type='Highest')
lowest_avg_demand = df_avg_demand_details_2017.iloc[[-1]].assign(demand_type='Lowest')

# Combine the highest and lowest entries into one DataFrame for display
df_combined = pd.concat([highest_avg_demand, lowest_avg_demand])

# Convert DataFrame to a list of tuples for the executemany insertion, including the new demand_type
data_to_insert = list(df_combined.itertuples(index=False, name=None))
c.executemany("""
INSERT INTO highest_lowest_avg_demand (weekday, month, season, avg_demand, demand_type) 
VALUES (?, ?, ?, ?, ?);
""", data_to_insert)

# Commit the transaction to the database
conn.commit()

# Use Pandas styling for display
styled_df_combined = df_combined.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print("Weekday, month, and season with the highest and lowest average demand rates in 2017:")
display(styled_df_combined)

# Close the database connection
conn.close()



Weekday, month, and season with the highest and lowest average demand rates in 2017:


,weekday,month,season,avg_demand,demand_type
0,Monday,June,summer,4.853999,Highest
59,Monday,January,spring,3.050786,Lowest


In [22]:
# TASK 7 subtask C

import sqlite3
import pandas as pd

# Assuming 'highest_avg_demand_weekday' is defined from the previous task
highest_avg_demand_weekday = 'Monday'  # Example, replace with actual variable from subtask B

# Connect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS hourly_demand_selected_weekday;")

# Create the table with specific column types
c.execute("""
CREATE TABLE hourly_demand_selected_weekday (
    hour TEXT,
    avg_demand REAL
);
""")

# Execute the query to calculate the average demand rate at different hours for the selected highest average demand weekday throughout 2017
c.execute(f"""
SELECT
    strftime('%H', timestamp) AS hour,
    AVG(demand) AS avg_demand
FROM CarSharing
WHERE 
    strftime('%Y', timestamp) = '2017' 
    AND strftime('%w', timestamp) = CASE '{highest_avg_demand_weekday}'
        WHEN 'Monday' THEN '1'
        WHEN 'Tuesday' THEN '2'
        WHEN 'Wednesday' THEN '3'
        WHEN 'Thursday' THEN '4'
        WHEN 'Friday' THEN '5'
    END
GROUP BY hour
ORDER BY avg_demand DESC
""")

hourly_demand_2017 = c.fetchall()

# Convert the result to a pandas DataFrame
df_hourly_demand_2017 = pd.DataFrame(hourly_demand_2017, columns=['hour', 'avg_demand'])

# Insert the DataFrame data into the SQLite database table using SQL commands
# Convert DataFrame to a list of tuples for the executemany insertion
data_to_insert = list(df_hourly_demand_2017.itertuples(index=False, name=None))
c.executemany("""
INSERT INTO hourly_demand_selected_weekday (hour, avg_demand) 
VALUES (?, ?);
""", data_to_insert)

# Commit the transaction to the database
conn.commit()

# Use Pandas styling for display
styled_df_hourly_demand_2017 = df_hourly_demand_2017.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print(f"\nAverage demand rate at different hours for {highest_avg_demand_weekday} throughout 2017:")
display(styled_df_hourly_demand_2017)

# Close the database connection
conn.close()



Average demand rate at different hours for Monday throughout 2017:


,hour,avg_demand
0,13,5.643554
1,12,5.621972
2,14,5.554613
3,15,5.515115
4,16,5.503753
5,11,5.437365
6,17,5.399252
7,10,5.223831
8,18,5.215943
9,19,4.990500


In [23]:
# TASK 7 subtask d.1

import sqlite3

# Reconnect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Query to find the most prevalent temp_category in 2017, excluding Saturdays and Sundays
c.execute("""
SELECT temp_category, COUNT(temp_category) AS count
FROM CarSharing
WHERE strftime('%Y', timestamp) = '2017'
AND strftime('%w', timestamp) NOT IN ('0', '6') -- Exclude Saturdays and Sundays
GROUP BY temp_category
ORDER BY count DESC
LIMIT 1
""")
most_prevalent_temp_category = c.fetchone()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS MostPrevalentTempCategory;")

# Create the table with specific column types
c.execute("""
CREATE TABLE MostPrevalentTempCategory (
    temp_category TEXT,
    occurrences INTEGER
);
""")

# Insert the result into the newly created table
c.execute("""
INSERT INTO MostPrevalentTempCategory (temp_category, occurrences) 
VALUES (?, ?);
""", (most_prevalent_temp_category[0], most_prevalent_temp_category[1]))

# Commit the transaction to the database
conn.commit()

# Close the database connection
conn.close()

# Print the result
print(f"Most prevalent temperature category in 2017 on weekdays: {most_prevalent_temp_category[0]} (Occurrences: {most_prevalent_temp_category[1]})")


Most prevalent temperature category in 2017 on weekdays: Mild (Occurrences: 1769)


In [24]:
# TASK 7 subtask d.2

import sqlite3

# Reconnect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Query to find the most prevalent weather condition in 2017, excluding Saturdays and Sundays
c.execute("""
SELECT w.weather, COUNT(*) AS count
FROM CarSharing cs
JOIN weather w ON cs.weather_code = w.weather_code
WHERE strftime('%Y', cs.timestamp) = '2017'
AND strftime('%w', cs.timestamp) NOT IN ('0', '6')  -- Exclude Saturdays and Sundays
GROUP BY w.weather
ORDER BY count DESC
LIMIT 1
""")
most_prevalent_weather = c.fetchone()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS MostPrevalentWeather;")

# Create the table with specific column types
c.execute("""
CREATE TABLE MostPrevalentWeather (
    weather TEXT,
    occurrences INTEGER
);
""")

# Insert the result into the newly created table
c.execute("""
INSERT INTO MostPrevalentWeather (weather, occurrences) 
VALUES (?, ?);
""", (most_prevalent_weather[0], most_prevalent_weather[1]))

# Commit the transaction to the database
conn.commit()

# Close the database connection
conn.close()

# Print the result
print(f"Most prevalent weather condition in 2017 on weekdays: {most_prevalent_weather[0]} (Occurrences: {most_prevalent_weather[1]})")




Most prevalent weather condition in 2017 on weekdays: Clear or partly cloudy (Occurrences: 2520)


In [25]:
# TASK 7 subtask d.3

import sqlite3
import pandas as pd

# Reconnect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Execute the query to analyze wind speed for each month in 2017, excluding Saturdays and Sundays
c.execute("""
SELECT 
    CASE strftime('%m', timestamp) 
        WHEN '01' THEN 'January' 
        WHEN '02' THEN 'February' 
        WHEN '03' THEN 'March' 
        WHEN '04' THEN 'April' 
        WHEN '05' THEN 'May' 
        WHEN '06' THEN 'June' 
        WHEN '07' THEN 'July' 
        WHEN '08' THEN 'August' 
        WHEN '09' THEN 'September' 
        WHEN '10' THEN 'October' 
        WHEN '11' THEN 'November' 
        WHEN '12' THEN 'December'
    END AS Month,
    AVG(windspeed) AS avg_wind_speed,
    MAX(windspeed) AS max_wind_speed,
    MIN(windspeed) AS min_wind_speed
FROM CarSharing
WHERE strftime('%Y', timestamp) = '2017'
AND strftime('%w', timestamp) NOT IN ('0', '6')  -- Exclude Saturdays and Sundays
GROUP BY Month
ORDER BY strftime('%m', timestamp)
""")
wind_speed_stats = c.fetchall()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS WindSpeedStats2017;")

# Create the table with specific column types
c.execute("""
CREATE TABLE WindSpeedStats2017 (
    Month TEXT,
    avg_wind_speed REAL,
    max_wind_speed REAL,
    min_wind_speed REAL
);
""")

# Insert the results into the newly created table
for row in wind_speed_stats:
    c.execute("""
    INSERT INTO WindSpeedStats2017 (Month, avg_wind_speed, max_wind_speed, min_wind_speed) 
    VALUES (?, ?, ?, ?);
    """, row)

# Commit the transaction to the database
conn.commit()

# Close the database connection
conn.close()

# Convert the results to a pandas DataFrame
df_wind_speed_stats = pd.DataFrame(wind_speed_stats, columns=['Month', 'Average Wind Speed', 'Maximum Wind Speed', 'Minimum Wind Speed'])

# Apply the center alignment style to each individual cell in the DataFrame
styled_df_wind_speed_stats = df_wind_speed_stats.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print("\nWind Speed Analysis for Each Month in 2017 (Excluding Weekends):")
styled_df_wind_speed_stats



Wind Speed Analysis for Each Month in 2017 (Excluding Weekends):


,Month,Average Wind Speed,Maximum Wind Speed,Minimum Wind Speed
0,January,14.476016,39.000700,0.000000
1,February,15.636455,51.998700,0.000000
2,March,15.597622,40.997300,0.000000
3,April,16.042593,40.997300,0.000000
4,May,12.615892,40.997300,0.000000
5,June,12.528143,35.000800,0.000000
6,July,12.406818,56.996900,0.000000
7,August,12.574349,43.000600,0.000000
8,September,11.957809,40.997300,0.000000
9,October,10.753541,32.997500,0.000000


In [26]:
# TASK 7 subtask d.4

import sqlite3
import pandas as pd

# Reconnect to the SQLite database for humidity analysis
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Execute the query to analyze humidity for each month in 2017, excluding Saturdays and Sundays
c.execute("""
SELECT 
    CASE strftime('%m', timestamp) 
        WHEN '01' THEN 'January' 
        WHEN '02' THEN 'February' 
        WHEN '03' THEN 'March' 
        WHEN '04' THEN 'April' 
        WHEN '05' THEN 'May' 
        WHEN '06' THEN 'June' 
        WHEN '07' THEN 'July' 
        WHEN '08' THEN 'August' 
        WHEN '09' THEN 'September' 
        WHEN '10' THEN 'October' 
        WHEN '11' THEN 'November' 
        WHEN '12' THEN 'December'
    END AS Month,
    AVG(humidity) AS avg_humidity,
    MAX(humidity) AS max_humidity,
    MIN(humidity) AS min_humidity
FROM CarSharing
WHERE strftime('%Y', timestamp) = '2017'
AND strftime('%w', timestamp) NOT IN ('0', '6')  -- Exclude Saturdays and Sundays
GROUP BY Month
ORDER BY strftime('%m', timestamp)
""")
humidity_stats = c.fetchall()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS HumidityStats2017;")

# Create the table with specific column types
c.execute("""
CREATE TABLE HumidityStats2017 (
    Month TEXT,
    avg_humidity REAL,
    max_humidity REAL,
    min_humidity REAL
);
""")

# Insert the results into the newly created table
for row in humidity_stats:
    c.execute("""
    INSERT INTO HumidityStats2017 (Month, avg_humidity, max_humidity, min_humidity) 
    VALUES (?, ?, ?, ?);
    """, row)

# Commit the transaction to the database
conn.commit()

# Close the database connection
conn.close()

# Convert the results to a pandas DataFrame
df_humidity_stats = pd.DataFrame(humidity_stats, columns=['Month', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity'])

# Apply the center alignment style to each individual cell in the DataFrame
styled_df_humidity_stats = df_humidity_stats.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print("\nHumidity Analysis for Each Month in 2017 (Excluding Weekends):")
styled_df_humidity_stats



Humidity Analysis for Each Month in 2017 (Excluding Weekends):


,Month,Average Humidity,Maximum Humidity,Minimum Humidity
0,January,55.826923,100.000000,28.000000
1,February,53.147541,100.000000,15.000000
2,March,54.524752,100.000000,0.000000
3,April,61.376206,100.000000,22.000000
4,May,70.955432,100.000000,24.000000
5,June,58.538710,94.000000,20.000000
6,July,62.110390,94.000000,33.000000
7,August,61.137313,94.000000,25.000000
8,September,75.495146,100.000000,48.000000
9,October,73.289552,100.000000,35.000000


In [88]:
# TASK 7 subtask d.5

import sqlite3
import pandas as pd

# Reconnect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
c = conn.cursor()

# Execute the query to find the average demand rate for each temperature category in 2017 on weekdays
c.execute("""
SELECT 
    temp_category, 
    AVG(demand) AS avg_demand
FROM CarSharing
WHERE 
    strftime('%Y', timestamp) = '2017'
    AND strftime('%w', timestamp) NOT IN ('0', '6')  -- Exclude Saturdays ('6') and Sundays ('0')
GROUP BY temp_category
ORDER BY avg_demand DESC
""")
avg_demand_by_temp_category = c.fetchall()

# Explicitly drop the table if it exists to ensure a fresh start
c.execute("DROP TABLE IF EXISTS AvgDemandByTempCategory;")

# Create the table with specific column types
c.execute("""
CREATE TABLE AvgDemandByTempCategory (
    temp_category TEXT,
    avg_demand REAL
);
""")

# Insert the results into the newly created table
for row in avg_demand_by_temp_category:
    c.execute("""
    INSERT INTO AvgDemandByTempCategory (temp_category, avg_demand) 
    VALUES (?, ?);
    """, row)

# Commit the transaction to the database
conn.commit()

# Close the database connection
conn.close()

# Convert the results to a pandas DataFrame
df_avg_demand_by_temp_category = pd.DataFrame(avg_demand_by_temp_category, columns=['Temperature Category', 'Average Demand'])

# Apply the center alignment style to each individual cell in the DataFrame
styled_df_avg_demand_by_temp_category = df_avg_demand_by_temp_category.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the styled DataFrame
print("\nAverage Demand Rate for Each Temperature Category in 2017:")
styled_df_avg_demand_by_temp_category



Average Demand Rate for Each Temperature Category in 2017:


,Temperature Category,Average Demand
0,Hot,4.730872
1,Mild,3.963292
2,Cold,3.235517


In [39]:
# # Task 7e
# import sqlite3
# import pandas as pd

# # Connect to the SQLite database
# conn = sqlite3.connect("CarSharing.db")
# cur = conn.cursor()

# # Drop the existing table to avoid an error if it already exists
# cur.execute("DROP TABLE IF EXISTS info_summary_demand2017")

# # Create the summary table for all months in 2017 with month names
# cur.execute("""
# CREATE TABLE info_summary_demand2017 AS
# SELECT 
#     CASE strftime('%m', timestamp)
#         WHEN '01' THEN 'January'
#         WHEN '02' THEN 'February'
#         WHEN '03' THEN 'March'
#         WHEN '04' THEN 'April'
#         WHEN '05' THEN 'May'
#         WHEN '06' THEN 'June'
#         WHEN '07' THEN 'July'
#         WHEN '08' THEN 'August'
#         WHEN '09' THEN 'September'
#         WHEN '10' THEN 'October'
#         WHEN '11' THEN 'November'
#         WHEN '12' THEN 'December'
#     END AS month,
#     AVG(demand) AS avg_demand,
#     AVG(windspeed) AS avg_windspeed,
#     MAX(windspeed) AS max_windspeed,
#     MIN(windspeed) AS min_windspeed,
#     AVG(humidity) AS avg_humidity,
#     MAX(humidity) AS max_humidity,
#     MIN(humidity) AS min_humidity
# FROM CarSharing
# WHERE strftime('%Y', timestamp) = '2017'
# GROUP BY month
# """)

# # Identify the month with the highest average demand in 2017
# cur.execute("""
# SELECT month
# FROM info_summary_demand2017
# ORDER BY avg_demand DESC
# LIMIT 1
# """)
# highest_demand_month_name = cur.fetchone()[0]

# # Fetch the detailed information for the month with the highest demand
# cur.execute("""
# SELECT * FROM info_summary_demand2017
# WHERE month = ?
# """, (highest_demand_month_name,))
# highest_month_info = cur.fetchall()

# # Fetch the detailed information for comparison to other months
# cur.execute("SELECT * FROM info_summary_demand2017")
# all_months_info = cur.fetchall()

# # Close the database connection
# conn.close()

# # Convert the results to pandas DataFrames for better visualization
# df_highest_month_info = pd.DataFrame(highest_month_info, columns=['Month', 'Average Demand', 'Average Wind Speed', 'Maximum Wind Speed', 'Minimum Wind Speed', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity']).set_index('Month')
# df_all_months_info = pd.DataFrame(all_months_info, columns=['Month', 'Average Demand', 'Average Wind Speed', 'Maximum Wind Speed', 'Minimum Wind Speed', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity']).set_index('Month')

# # Styling the DataFrames
# styled_highest_month_info = df_highest_month_info.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
#     'selector': 'th',
#     'props': [('text-align', 'center')]
# }])
# styled_all_months_info = df_all_months_info.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
#     'selector': 'th',
#     'props': [('text-align', 'center')]
# }])

# # Display the results with styling
# print(f"Information for {highest_demand_month_name} (Month with the Highest Demand):")
# display(styled_highest_month_info)
# print("\nInformation Summary for All Months in 2017:")
# display(styled_all_months_info)

import sqlite3
import pandas as pd

# Connect to the SQLite database
conn = sqlite3.connect("CarSharing.db")
cur = conn.cursor()

# Drop the existing table to avoid an error if it already exists
cur.execute("DROP TABLE IF EXISTS info_summary_demand2017")

# Create the summary table for all months in 2017 with month names
cur.execute("""
CREATE TABLE info_summary_demand2017 AS
SELECT 
    CASE strftime('%m', timestamp)
        WHEN '01' THEN 'January'
        WHEN '02' THEN 'February'
        WHEN '03' THEN 'March'
        WHEN '04' THEN 'April'
        WHEN '05' THEN 'May'
        WHEN '06' THEN 'June'
        WHEN '07' THEN 'July'
        WHEN '08' THEN 'August'
        WHEN '09' THEN 'September'
        WHEN '10' THEN 'October'
        WHEN '11' THEN 'November'
        WHEN '12' THEN 'December'
    END AS month,
    AVG(demand) AS avg_demand,
    AVG(windspeed) AS avg_windspeed,
    MAX(windspeed) AS max_windspeed,
    MIN(windspeed) AS min_windspeed,
    AVG(humidity) AS avg_humidity,
    MAX(humidity) AS max_humidity,
    MIN(humidity) AS min_humidity
FROM CarSharing
WHERE strftime('%Y', timestamp) = '2017'
GROUP BY month
""")

# Identify the month with the highest average demand in 2017
cur.execute("""
SELECT month
FROM info_summary_demand2017
ORDER BY avg_demand DESC
LIMIT 1
""")
highest_demand_month_name = cur.fetchone()[0]

# Fetch the detailed information for the month with the highest demand
cur.execute("""
SELECT * FROM info_summary_demand2017
WHERE month = ?
""", (highest_demand_month_name,))
highest_month_info = cur.fetchall()

# Fetch the detailed information for comparison to other months
cur.execute("SELECT * FROM info_summary_demand2017")
all_months_info = cur.fetchall()

# Close the database connection
conn.close()

# Convert the results to pandas DataFrames for better visualization
df_highest_month_info = pd.DataFrame(highest_month_info, columns=['Month', 'Average Demand', 'Average Wind Speed', 'Maximum Wind Speed', 'Minimum Wind Speed', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity']).set_index('Month')
df_all_months_info = pd.DataFrame(all_months_info, columns=['Month', 'Average Demand', 'Average Wind Speed', 'Maximum Wind Speed', 'Minimum Wind Speed', 'Average Humidity', 'Maximum Humidity', 'Minimum Humidity']).set_index('Month')

# Styling the DataFrames
styled_highest_month_info = df_highest_month_info.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])
styled_all_months_info = df_all_months_info.style.set_properties(**{'text-align': 'center'}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center')]
}])

# Display the results with styling
print(f"Information for {highest_demand_month_name} (Month with the Highest Demand):")
display(styled_highest_month_info)
print("\nInformation Summary for All Months in 2017:")
display(styled_all_months_info)



Information for July (Month with the Highest Demand):


,Average Demand,Average Wind Speed,Maximum Wind Speed,Minimum Wind Speed,Average Humidity,Maximum Humidity,Minimum Humidity
Month,,,,,,,
July,4.787655,12.015846,56.996900,0.000000,60.292035,94.000000,17.000000



Information Summary for All Months in 2017:


,Average Demand,Average Wind Speed,Maximum Wind Speed,Minimum Wind Speed,Average Humidity,Maximum Humidity,Minimum Humidity
Month,,,,,,,
April,4.049236,15.852275,40.997300,0.000000,66.248899,100.000000,22.000000
August,4.642341,12.411122,43.000600,0.000000,62.173626,94.000000,25.000000
December,4.276869,10.836460,43.000600,0.000000,65.180617,100.000000,26.000000
February,3.679483,15.577717,51.998700,0.000000,53.580717,100.000000,8.000000
January,3.388312,13.748052,39.000700,0.000000,56.307692,100.000000,28.000000
July,4.787655,12.015846,56.996900,0.000000,60.292035,94.000000,17.000000
June,4.723880,11.827618,35.000800,0.000000,58.370861,100.000000,20.000000
March,3.745415,15.974884,40.997300,0.000000,55.997753,100.000000,0.000000
May,4.571585,12.427391,40.997300,0.000000,71.371429,100.000000,24.000000
